In [110]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import logging
import seaborn as sns
from sklearn.metrics import mean_squared_error
from ydata_profiling import ProfileReport
from IPython.display import display
from libs.logger import configure_logging
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score
import os
# TODO: Documentación completa de la solución.

In [111]:
configure_logging()
log = logging.getLogger(__name__)
log.debug("testing the debug message")
# esto es para ocultar algunos mensajes de log de librerías como matplotlib
logging.getLogger("matplotlib").setLevel(logging.WARNING)

2025-05-20 14:09:43,789 [   DEBUG] __main__:3 (15544/MainThread)  - testing the debug message


In [112]:
# Carga del dataset en un dataframe y visualizacion de los primeros 5 registros.
df = pd.read_csv("../data/winequality-red.csv", delimiter=";")
print(np.shape(df))  # este comando imprime la forma del dataframe

(1599, 12)


In [113]:
# Generación del reporte y guardado en un archivo HTML en la carpeta output.
#profile = ProfileReport(df, title="Red wine quality Report")
#profile.to_file("../output/red-wine-report.html")

A modo de recomendación del profesor, se tienen las siguientes consideraciones:
* Maneje los datos faltantes, si los hay. (con el reporte de ydata-profiling podemos ver que no hay datos faltantes)
* Convierta la variable objetivo "quality" en una variable binaria: "low" (puntuación de calidad menor a 7) y "high" (puntuación de calidad mayor o igual a 7)
* Divida los datos en conjuntos de entrenamiento y prueba

In [114]:
# Convertir la columna quality a binaria considerando como valor de corte 7, siendo menor a 7 'low' y mayor igual 7 'high'.
# Primero hay que ver el tipo de variables que manejan.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [115]:
# Crear la variable binaria 'quality_label'
df['quality_label'] = df['quality'].apply(lambda x: 'low' if x < 7 else 'high')

# Verificar la distribución de la nueva columna generada
print(df['quality_label'].value_counts())

quality_label
low     1382
high     217
Name: count, dtype: int64


In [116]:
X = df.drop(['quality', 'quality_label'], axis=1)
y = df['quality_label']

# Codificar la variable objetivo (Naïve Bayes necesita etiquetas numéricas)

le = LabelEncoder()
y_encoded = le.fit_transform(y)  # low=0, high=1

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded)

In [117]:
# Instanciar el modelo
nb_model = GaussianNB()

# Entrenar el modelo
nb_model.fit(X_train, y_train)

GaussianNB()

In [118]:
# Realizar predicciones
y_pred = nb_model.predict(X_test)

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", conf_matrix)

# Métricas
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {acc:.2f}")
print(f"Precision: {prec:.2f}")
print(f"Recall: {rec:.2f}")
print(f"F1-score: {f1:.2f}")


Matriz de confusión:
 [[ 44  21]
 [ 46 369]]
Accuracy: 0.86
Precision: 0.95
Recall: 0.89
F1-score: 0.92


In [119]:
# Obtener probabilidades
y_proba = nb_model.predict_proba(X_test)[:, 1]  # Probabilidad de clase 'high'

# Curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)

# Crear la carpeta ../output si no existe
output_dir = "../output"
os.makedirs(output_dir, exist_ok=True)

# Ruta del archivo a guardar
output_path = os.path.join(output_dir, "roc_curve.png")

# Graficar y guardar la curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC - Clasificador Naïve Bayes')
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig(output_path)
plt.close()

print(f"Curva ROC guardada en: {output_path}")

Curva ROC guardada en: ../output\roc_curve.png
